
# Diseño de software para cómputo científico

----

## Unidad 1: Meta programación


### Agenda de la Unidad 1
---

- Clase 1:
    - Diferencias entre alto y bajo nivel.
    - Lenguajes dinámicos y estáticos.
    
- Limbo:
    - Introducción al lenguaje Python.
    - Librerías de cómputo científico.
    
- Clase Limbo + 1 y Limbo + 2:
    - **Orientación a objetos**, decoradores.

### Clases abstractas

In [1]:
from collections.abc import Mapping

class MyDict(Mapping):
    ...

In [2]:
MyDict()

TypeError: Can't instantiate abstract class MyDict with abstract methods __getitem__, __iter__, __len__

In [21]:
from collections.abc import Mapping

class MyDict(Mapping):
    def __init__(self, **kwargs):
        self.d = kwargs
        
    def __getitem__(self, k):
        return self.d[k]
    
    def __iter__(self):
        return iter(self.d)
    
    def __len__(self):
        return len(self.d)
    def __getattr__(self, a):
        return self[a]

d = MyDict(a= 42)
d.a

42

### Clases abstractas

In [22]:
import abc

class ClaseBase(metaclass=abc.ABCMeta):
    
    @abc.abstractmethod
    def m(self):
        raise NotImplementedError()       


In [23]:
ClaseBase()

TypeError: Can't instantiate abstract class ClaseBase with abstract methods m

In [26]:
class Clase(ClaseBase):
    def m(self):
        return 42

Clase().m()

42

## Funciones de meta programación
----

In [27]:
class P: ...

In [28]:
p = P()
setattr(p, "var", 42)
p.var

42

In [29]:
getattr(p, "var")

42

In [30]:
hasattr(p, "var")

True

In [31]:
delattr(p, "var")
getattr(p, "var", "Not-found")

'Not-found'

## Creando clases con type
-----

In [69]:
def init(self, b):
    self.b = b

MyClass = type('MyClass',(dict,),{'a':True, "__init__": init})

del init

In [70]:
class MyClass(dict, metaclass=type):
    a = True
    def __init__(self, b):
        self.b = b

## Meta clases
---
Una meta clase es una clase que crea clases

In [56]:
class DataClassMeta(type):
    
    def __init__(cls, name, bases, members):
        cls._init_params = {}
        for k, v in members.items():
            if not k.startswith("__"):
                delattr(cls, k)
                cls._init_params[k] = v    

In [57]:
class DataClass(metaclass=DataClassMeta):
    
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            if k not in self._init_params:
                raise ValueError(k)
            elif self._init_params[k] != type(v):
                raise TypeError(f"{k} must be instance of {self._init_params[k]}")
            setattr(self, k, v)

In [59]:
class MyClass(DataClass):
    a = int
    b = str

MyClass(a=1., b="ll")

TypeError: a must be instance of <class 'int'>

## Meta clases - `__init_subclass__`

Este método se llama siempre que la clase contenedora se hereda. cls es entonces la nueva subclase. Si se define como un método de instancia normal, este método se convierte implícitamente en un método de clase (osea se le pone de prepo el `@classmethod`).

In [66]:
class DataClass2:
    
    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        members = dict(vars(cls))
        cls._init_params = {}
        for k, v in members.items():
            if not k.startswith("__"):
                delattr(cls, k)
                cls._init_params[k] = v 
    
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            if k not in self._init_params:
                raise ValueError(k)
            elif self._init_params[k] != type(v):
                raise TypeError(f"{k} must be instance of {self._init_params[k]}")
            setattr(self, k, v)

In [67]:
class MyClass2(DataClass2):
    a = int
    b = str

MyClass(a=1, b="ll")